<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_overlap10_mixed_pubmed_bigtweet_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient_overlap_10.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,3,cracking sensations in my joints
2,4,purple sploches around waist
3,5,fast and slow heartbeat
4,5,increased heart rate


In [14]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_askpatient.csv')

In [15]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [17]:
data_clas.add_test(test,label=0)


In [18]:
data_lm.save()
data_clas.save()

In [19]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'a',
 'muscle',
 'with',
 'for',
 'was',
 'diclofenac',
 'were',
 'severe',
 'on',
 'my',
 'is',
 'myalgia',
 'lower',
 'fatigue',
 'rt',
 'by',
 'loss',
 'limb',
 'sodium',
 'weakness',
 'cramp',
 'at',
 'disease',
 'that',
 'stomach',
 'or',
 'as',
 'depression',
 'back',
 '-',
 'patients',
 'from',
 'i',
 'this',
 'not',
 'cramps',
 'upper',
 'gas',
 'joint',
 'drug',
 'memory',
 'leg',
 'you',
 'mg',
 'all',
 'legs',
 'pains',
 'headache',
 'knee',
 'be',
 'are',
 'group',
 'unable',
 'shoulder',
 'study',
 'gastrointestinal',
 'arthralgia',
 'treatment',
 'neck',
 'excessive',
 'foot',
 'arthritis',
 'after',
 'it',
 'an',
 'aches',
 'hip',
 'amp',
 'have',
 'like',
 'voltaren',
 'numbness',
 'insomnia',
 'arthrotec',
 'release',
 'no',
 'increased',
 'p',
 'bleeding',
 'time',
 'nausea',
 'both',
 '/',
 'gluten',
 'swelling',
 'can',

In [20]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [21]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.848570,3.467800,0.446479,35:01


In [23]:
# learn.fit_one_cycle(1, 1e-2)

In [24]:
# learn.fit_one_cycle(1, 1e-3)

In [25]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.518413,3.406411,0.452141,35:04


In [26]:
learn.save_encoder('pubmed_bigtweet_askpatient_mixed_overlap10_first')

In [27]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_bigtweet_askpatient_mixed_overlap10_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.933878,3.821103,0.345562,01:13


In [28]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.888889,2.975978,0.494675,01:25


In [29]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.306977,2.609479,0.552663,03:31


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.088660,2.293450,0.609467,03:23
1,1.695000,2.141110,0.627219,03:22


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.516374,1.978610,0.653254,03:10
1,1.330484,1.853077,0.679290,03:32


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.202429,1.741072,0.701775,03:28
1,1.008164,1.639205,0.708876,03:14


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.955702,1.566788,0.718343,03:17
1,0.839721,1.554028,0.725444,03:12


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.792176,1.475052,0.728994,03:15
1,0.694485,1.451819,0.744379,03:12


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.656207,1.418384,0.747929,03:07
1,0.624535,1.398026,0.749112,03:21
2,0.555884,1.366051,0.763314,03:18
3,0.460025,1.335264,0.772781,03:33


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.431328,1.353074,0.778698,03:31
1,0.496206,1.366226,0.760947,03:12
2,0.410655,1.264189,0.775148,03:28
3,0.380106,1.263701,0.778698,03:38


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.407049,1.277078,0.778698,03:39
1,0.388038,1.249681,0.788166,03:14
2,0.351847,1.212389,0.796450,03:31
3,0.306422,1.238016,0.797633,03:13


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.313325,1.228012,0.796450,03:23
1,0.371851,1.208787,0.798817,03:21
2,0.276819,1.216164,0.801183,03:22
3,0.268749,1.214440,0.802367,03:25


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.240111,1.265099,0.794083,03:19
1,0.286093,1.258279,0.807101,03:25
2,0.245082,1.241860,0.804734,03:30
3,0.221645,1.255758,0.803550,03:13


In [40]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [41]:
# learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

In [42]:
# learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [43]:
# learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

In [44]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.252074,1.350192,0.808284,03:13


In [45]:
# save the best model

learn.save_encoder('pubmed_bigtweet_askpatient_mixed_overlap10')

# Part three: Predict on the test dataset

In [46]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [47]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

681
681
[0, 467, 61, 334, 5, 1026, 646, 472, 10, 447, 788, 374, 135, 27, 707, 532, 706, 614, 2, 30, 32, 34, 741, 37, 38, 623, 41, 1002, 983, 983, 325, 100, 100, 640, 61, 464, 71, 71, 71, 76, 352, 182, 1009, 79, 109, 109, 109, 109, 109, 129, 37, 109, 569, 80, 794, 89, 892, 557, 850, 94, 94, 448, 734, 546, 683, 98, 244, 338, 100, 100, 100, 100, 61, 100, 405, 236, 107, 108, 108, 892, 35, 313, 211, 484, 172, 281, 108, 190, 730, 35, 120, 853, 628, 125, 339, 125, 127, 698, 129, 129, 700, 132, 607, 142, 144, 794, 794, 182, 354, 157, 509, 165, 81, 405, 194, 121, 412, 860, 182, 182, 182, 182, 183, 183, 187, 1026, 1028, 190, 190, 190, 190, 190, 190, 190, 190, 197, 34, 34, 34, 34, 805, 568, 2, 210, 210, 210, 210, 210, 211, 34, 257, 214, 214, 351, 214, 214, 339, 448, 220, 44, 228, 228, 79, 228, 228, 228, 228, 776, 541, 234, 235, 236, 314, 402, 339, 1009, 910, 187, 820, 448, 1009, 1009, 390, 448, 638, 245, 245, 245, 245, 245, 245, 467, 246, 646, 35, 390, 715, 259, 260, 260, 390, 265, 419, 660, 402,

In [48]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

302
0.4434654919236417
